In [ ]:
import math

import pandas
import pandas as pd
import transbigdata as tbd
import datetime
import json
import os

In [ ]:
pd.set_option('display.max_columns', 1000)
pd.set_option('display.min_rows', 50)
pd.set_option('expand_frame_repr', False)

for file in os.listdir('新数据'):
    origin = pd.read_csv('新数据/' + file, names=['car_id', 'time', 'car_state', 'charged_state', 'run_model', 'speed', 'sum_km', 'sum_V', 'sum_A', 'SOC', 'DC-DC', 'gear', 'kOhm', 'lon', 'lat', 'max_V', 'min_V', 'max_temp', 'min_temp', 'motor_seq', 'motor_speed', 'motor_tor'])
    break

# origin = pd.read_csv('04.csv')
# origin.columns = ['id', 'car_id', 'time', 'car_state', 'charged_state', 'run_model', 'speed', 'sum_km', 'sum_V', 'sum_A', 'SOC', 'DC-DC', 'gear', 'kOhm', 'lon', 'lat', 'max_V', 'min_V', 'max_temp', 'min_temp', 'motor_seq', 'motor_speed', 'motor_tor']
# del origin['id']

origin

In [ ]:
# 数据清洗
data = origin.drop(columns = ['car_state', 'run_model', 'DC-DC', 'gear', 'kOhm', 'speed', 'sum_km', 'sum_V', 'sum_A', 'max_V', 'min_V', 'max_temp', 'min_temp', 'motor_seq', 'motor_speed', 'motor_tor'], axis=1)

# 剔除范围外数据
data = data[data['lon'] > 120]
data = data[data['lon'] < 123]
data = data[data['lat'] > 30]
data = data[data['lat'] < 32]

data['SOC_last'] = data['SOC'].shift()
data = data[data['SOC'] - data['SOC_last'] < 2]
data = data[data['SOC'] - data['SOC_last'] > -2]
del data['SOC_last']

# data['charged_state'] = data['charged_state'].apply(lambda _: 3 if _ == 2 else _)

# car_id 设为简单的1，2，3……
data['car_id_next'] = data['car_id'].shift()
data['car_id_alter'] = data.apply(lambda _: _['car_id'] != _['car_id_next'], axis=1)
data['_'] = data['car_id_alter'].cumsum()
data.drop(['car_id', 'car_id_next', 'car_id_alter'], axis=1, inplace=True)
data.rename(columns={'_': 'car_id'}, inplace=True)

# 时间str转datetime
data['time'] = data['time'].apply(lambda _: datetime.datetime.strptime(_, '%Y-%m-%d %H:%M:%S'))

data

In [ ]:
# 计算 poschange 和 posid

data['lon_last'] = data['lon'].shift()
data['lat_last'] = data['lat'].shift()

data['poschange'] = data.apply(lambda _: int(_['lon'] == _['lon_last'] and _['lat'] == _['lat_last']), axis=1)
data['pcs'] = data['poschange'].shift(-1)
data['abs'] = data.apply(lambda _: abs(_['pcs'] - _['poschange']), axis=1)
data['posid'] = data.groupby(['car_id'])['abs'].cumsum()
data.drop(['lon_last', 'lat_last', 'pcs', 'abs'], axis=1, inplace=True)
data.dropna(inplace=True)
data

In [ ]:
# group by

def count_dis_by_tbd(df):
    df['lon_last'] = df['lon'].shift()
    df['lat_last'] = df['lat'].shift()
    df.dropna(inplace=True)
    return int(sum([tbd.getdistance(__['lon'], __['lat'], __['lon_last'], __['lat_last']) for __ in df.iloc]))

gb = data.groupby(['car_id', 'posid'])
data2 = pd.concat([gb['time'].apply(lambda _: _.iloc[0]), gb['time'].apply(lambda _: _.iloc[-1]),
                   gb['SOC'].apply(lambda _: _.iloc[0]),  gb['SOC'].apply(lambda _: _.iloc[-1]),
                   gb['lon'].apply(lambda _: _.iloc[0]),  gb['lat'].apply(lambda _: _.iloc[0]),
                   gb['lon'].apply(lambda _: _.iloc[-1]), gb['lat'].apply(lambda _: _.iloc[-1]), gb.apply(count_dis_by_tbd)], axis=1)
data2.columns = ['time_start', 'time_end', 'SOC_start', 'SOC_end', 'lon_start', 'lat_start', 'lon_end', 'lat_end', 'dis_sum']
data2.reset_index(inplace=True)
data2

In [ ]:
# drop_duplicates

data3 = data.drop_duplicates(subset=['car_id', 'posid'])
data3 = data3.drop(['time', 'SOC', 'lon', 'lat'], axis=1)
data3

In [ ]:
# merge

data4 = pd.merge(left=data2, right=data3)
data4

In [ ]:
min30 = pd.Timedelta(minutes=30)
data666 = data4.copy()
data666['new_poschange'] = data666.apply(lambda _: 0 if _['time_end'] - _['time_start'] < min30 else 1 - _['poschange'], axis=1)
data666.rename(columns={'time_start' : 'time'}, inplace=True)
data666.drop(['poschange', 'posid', 'SOC_start', 'SOC_end', 'lon_start', 'lat_start', 'lon_end', 'lat_end', 'charged_state', 'dis_sum'], axis=1, inplace=True)
data666

In [ ]:
data777 = pd.merge(data, data666, on=['car_id', 'time'], how='outer')
data777

In [ ]:
last_new_poschange = data777['new_poschange'][0]
last_time_end = data777['time_end'][0]
# min1 = pd.Timedelta(minutes=1)

def update(time, poschange, new_poschange, time_end):
    global last_new_poschange
    global last_time_end
    if not math.isnan(new_poschange):
        last_new_poschange = new_poschange
        last_time_end = time_end # + min1
        return new_poschange
    elif time <= last_time_end:
        return last_new_poschange
    else:
        return poschange

data888 = data777.copy()

data888['SOC_last'] = data888['SOC'].shift()
data888['SOC_increase'] = data888.apply(lambda _: int(_['SOC'] > _['SOC_last']), axis=1)

# data888['poschange'] = data888.apply(lambda _: _['new_poschange'] if not math.isnan(_['new_poschange']) else _['poschange'], axis=1)
data888['poschange'] = data888.apply(lambda _: update(_['time'], _['poschange'], _['new_poschange'], _['time_end']), axis=1)
data888['pcs'] = data888['poschange'].shift(-1)
data888['abs'] = data888.apply(lambda _: abs(_['pcs'] - _['poschange']), axis=1).shift()
data888['posid'] = data888.groupby(['car_id'])['abs'].cumsum()
# data888['posid'][0] = 0
data888.drop(['pcs', 'abs'], axis=1, inplace=True)
data888

In [ ]:
def count_dis_by_tbdd(df):
    df['lon_last'] = df['lon'].shift()
    df['lat_last'] = df['lat'].shift()
    df.dropna(subset=['lon', 'lat', 'lon_last', 'lat_last'], inplace=True)
    return int(sum([tbd.getdistance(__['lon'], __['lat'], __['lon_last'], __['lat_last']) for __ in df.iloc]))

def charge_start_and_end(df):
    _ = df.loc[df.SOC_increase == 1, 'time']
    return None if _.empty else (_.iat[0], _.iat[-1])

gb = data888.groupby(['car_id', 'posid'])
data22 = pd.concat([gb['time'].apply(lambda _: _.iloc[0]), gb['time'].apply(lambda _: _.iloc[-1]),
                   gb['SOC'].apply(lambda _: _.iloc[0]),  gb['SOC'].apply(lambda _: _.iloc[-1]),
                   gb['lon'].apply(lambda _: _.iloc[0]),  gb['lat'].apply(lambda _: _.iloc[0]),
                   gb['lon'].apply(lambda _: _.iloc[-1]), gb['lat'].apply(lambda _: _.iloc[-1]), gb.apply(count_dis_by_tbdd), gb.apply(charge_start_and_end)], axis=1)
data22.columns = ['time_start', 'time_end', 'SOC_start', 'SOC_end', 'lon_start', 'lat_start', 'lon_end', 'lat_end', 'dis_sum', 'charge_start_and_end']
data22['charge_start_and_end'] = data22.apply(lambda _: _['charge_start_and_end'] if _['dis_sum'] == 0 else None, axis=1)
data22.reset_index(inplace=True)
data22

In [ ]:
data33 = data888.drop(columns=['time_end', 'new_poschange', 'time', 'SOC', 'SOC_last', 'lon', 'lat', 'SOC_increase'])
data33 = data33.drop_duplicates(subset=['car_id', 'posid']).dropna()
data33

In [ ]:
data44 = pd.merge(left=data22, right=data33)
data44

In [ ]:
# 计算 time_duration

def count_time_duration(_):
    total_sec = (_['time_end'] - _['time_start']).total_seconds()
    return total_sec // 3600, (total_sec % 3600) // 60, total_sec % 60

data5 = data44.copy()
data5['time_duration_h_m_s'] = data5.apply(count_time_duration, axis=1)
data5

In [ ]:
# charge_state： 1充电； 3不充电

In [ ]:
os.makedirs('./data_csv/', exist_ok=True)
os.makedirs('./data_json', exist_ok=True)

In [ ]:
run = data5[data5['dis_sum'] != 0]
run.reset_index(inplace=True)
del run['charged_state']
del run['poschange']
del run['index']
run.to_csv('./data_csv/data_run.csv')
tbd.dumpjson(run.to_json(orient="records"), './data_json/data_run.json')
run

In [ ]:
stop = data5[data5['dis_sum'] == 0].copy()
stop.reset_index(inplace=True)
stop.drop(columns = ['poschange', 'dis_sum', 'index', 'lon_end', 'lat_end', 'charged_state'], axis=1, inplace=True)
stop.rename(columns={'lon_start':'lon','lat_start':'lat'}, inplace=True)
stop.to_csv('./data_csv/data_stop.csv')
tbd.dumpjson(stop.to_json(orient="records"), './data_json/data_stop.json')
stop

In [ ]:
def count_charge_duration(_):
    total_sec = (_['time_end_charge'] - _['time_start_charge']).total_seconds()
    return total_sec // 3600, (total_sec % 3600) // 60, total_sec % 60


charged = stop[stop['SOC_start'] < stop['SOC_end']].copy()
charged[['time_start_charge', 'time_end_charge']] = charged['charge_start_and_end'].apply(pd.Series)
del charged['charge_start_and_end']
charged['charge_duration_h_m_s'] = charged.apply(count_charge_duration, axis=1)
charged.to_csv('./data_csv/data_charged.csv')
tbd.dumpjson(charged.to_json(orient="records"), './data_json/data_charged.json')
charged

In [ ]:
stop_xy = stop[['lon', 'lat']].copy()
stop_xy.plot.scatter('lon', 'lat')

In [ ]:
charged_xy = charged[['lon', 'lat']].copy()
charged_xy.plot.scatter('lon', 'lat')

In [ ]:
chag_rate = charged.drop(columns = ['car_id', 'posid', 'time_start', 'time_end', 'time_duration_h_m_s'], axis=1).copy()
chag_rate['start_h'] = chag_rate['time_start_charge'].apply(lambda _: _.hour)
chag_rate['start_m'] = chag_rate['time_start_charge'].apply(lambda _: _.minute)
chag_rate['end_h'] = chag_rate['time_end_charge'].apply(lambda _: _.hour)
chag_rate['end_m'] = chag_rate['time_end_charge'].apply(lambda _: _.minute)
chag_rate['charge_rate'] = chag_rate.apply(lambda _: (_.SOC_end - _.SOC_start) / (_.charge_duration_h_m_s[0] + _.charge_duration_h_m_s[1] / 60), axis=1)
# charge_rate: SOC per hour 假设所有车辆电池大小相同，以后可引入电池大小数据
chag_rate

In [ ]:
# c1 = chag_rate[chag_rate['start_h'] > chag_rate['end_h']].copy()
# c2 = chag_rate[chag_rate['start_h'] <= chag_rate['end_h']].copy()
# c1['end_h'] += 24
# chag_rate = pd.concat([c1, c2], axis=0)



# statistics_charged_by_hour = [0] * 24
# for i in range(len(chag_rate)): # 对小时循环
#     _ = chag_rate.iloc[i]
#     _cr = _.charge_rate
#     statistics_charged_by_hour[_.start_h] += (60 - _.start_m) * _cr
#     statistics_charged_by_hour[_.start_h+1:_.end_h+1] = [_ + 60 * _cr for _ in statistics_charged_by_hour[_.start_h+1:_.end_h+1]]
#     statistics_charged_by_hour[_.end_h] += _.end_m * _cr
#
# statistics_charged_by_hour = [_ / 60 for _ in statistics_charged_by_hour]
# for _ in range(24):
#     print("".join([str(_), "点到", str(_+1), "点，充入电量百分比为：\t"]), statistics_charged_by_hour[_])



# scbh2 = [0] * 24
# for i in range(24):

# chag_rate[chag_rate['start_h'] < 20 & chag_rate['end_h'] > 21]
# chag_rate[chag_rate['start_h'] > 20]['charge_rate'].sum()

# chag_rate.apply(lambda _: _['start_h'] < 24 < _['end_h'], axis=1)

In [ ]:
stop_not_charged = stop[stop['SOC_start'] >= stop['SOC_end']]
del stop_not_charged['charge_start_and_end']
stop_not_charged

In [ ]:
stop_over_1h_not_charged = stop_not_charged[stop_not_charged['time_duration_h_m_s'].apply(lambda _: _[0] > 0)]
stop_over_1h_not_charged

In [ ]:
stop_over_1h___not_charged___SOC_less_than_40 = stop_over_1h_not_charged[stop_over_1h_not_charged['SOC_start'] < 40]
stop_over_1h___not_charged___SOC_less_than_40.to_csv('./data_csv/data_potential_charging_requirements.csv')
tbd.dumpjson(stop_over_1h___not_charged___SOC_less_than_40.to_json(orient="records"), './data_json/data_potential_charging_requirements.json')
stop_over_1h___not_charged___SOC_less_than_40